## Appendix A. Data Wrangling

In [4]:
#preliminaries

import os
from bs4 import BeautifulSoup
import requests
import time
import re
import numpy as np
import pandas as pd
import sqlite3

### Appendix A.1. Stackexchange Archive

The dataset used for this analysis is collected from the stackexchange repository: https://archive.org/details/stackexchange. Specifically, the file stackoverflow.com-Posts.7z which contains an XML file of all posts in stackoverflow was downloaded. Due to its large size (13.6G), the file was split into multiple files using `split -l` from `git bash`.

The XML file contains millions of rows where one row would contain the following information:

```xml
  <row Id="4" PostTypeId="1" AcceptedAnswerId="7" CreationDate="2008-07-31T21:42:52.667" Score="617" ViewCount="40863" Body="&lt;p&gt;I want to use a track-bar to change a form's opacity.&lt;/p&gt;&#xA;&#xA;&lt;p&gt;This is my code:&lt;/p&gt;&#xA;&#xA;&lt;pre&gt;&lt;code&gt;decimal trans = trackBar1.Value / 5000;&#xA;this.Opacity = trans;&#xA;&lt;/code&gt;&lt;/pre&gt;&#xA;&#xA;&lt;p&gt;When I build the application, it gives the following error:&lt;/p&gt;&#xA;&#xA;&lt;blockquote&gt;&#xA;  &lt;p&gt;Cannot implicitly convert type &lt;code&gt;'decimal'&lt;/code&gt; to &lt;code&gt;'double'&lt;/code&gt;.&lt;/p&gt;&#xA;&lt;/blockquote&gt;&#xA;&#xA;&lt;p&gt;I tried using &lt;code&gt;trans&lt;/code&gt; and &lt;code&gt;double&lt;/code&gt; but then the control doesn't work. This code worked fine in a past VB.NET project.&lt;/p&gt;&#xA;" OwnerUserId="8" LastEditorUserId="6786713" LastEditorDisplayName="Rich B" LastEditDate="2018-07-02T17:55:27.247" LastActivityDate="2019-01-17T13:39:48.937" Title="Convert Decimal to Double?" Tags="&lt;c#&gt;&lt;floating-point&gt;&lt;type-conversion&gt;&lt;double&gt;&lt;decimal&gt;" AnswerCount="13" CommentCount="1" FavoriteCount="46" CommunityOwnedDate="2012-10-31T16:42:47.213" />
```

Each subfile (from the large file) would then be processed into an SQL database using `projdbload.py` with the code shown below:

```python
import sqlite3
import psycopg2
import pandas as pd
import re
import os
from xml.etree import ElementTree
import click
import random


@click.group()
def main():
    pass


@main.command('pathdbload')
@click.argument('pathname', type=str)
@click.argument('tablename', type=str)
@click.argument('database', type=str)
@click.option('--filt', type=str, default='row', help='filter')
def FiletoDBLoad(pathname, tablename, database, filt='row'):
    """Load the XML files under `pathname` and store the attributes in the
    table `tablename` under `database`.
    
    Parameters
    ----------
    pathname : str
        filepath leading to directory containing the XML files to be processed
    tablename : str
        name of the SQL table to store the data in
    database : str
        filename of the SQL database
    filt : str, optional
        filters the XML file based on the filt query. default value is row
    """
    files = []
    
    # Get the files under the directory "pathname"
    for r, d, f in os.walk(pathname):
        for file in f:
            os.system('echo "Will process this file "'+file)
            files.append(os.path.join(r, file))

    # Initialize the database
    conn = sqlite3.connect(database)
    curs = conn.cursor()

    for filename in files:
        # Store the current file into a temp file
        os.system('echo "Processing File "'+filename)
        os.system(
            'echo "<?xml version=\\\"1.0\\\" encoding=\\\"utf-8\\\"?>" > .temp.txt')
        head = '"<'+tablename.lower()+'>"'
        os.system('echo '+head+' >> .temp.txt')
        # Filter the XML file according to the query filt
        os.system('cat '+filename+' |grep "'+filt.replace('"', r'\"') +
                  '" >> .temp.txt')
        tail = '"</'+tablename.lower()+'>"'
        os.system('echo '+tail+' >> .temp.txt')

        # Open the temp file
        with open('.temp.txt', 'r', encoding='utf-8') as f:

            lines = f.read().splitlines()
            flag = True
            # Get the minimum number of columns to use as header from 5 sampled rows/elements
            for i in random.sample(range(2, len(lines)), 5):
                header_temp = lines[i]
                # Since display name and user ID are both unique identifier of users:
                header_temp = header_temp.replace('DisplayName', 'UserId')
                header_temp = re.findall(r'(\S*?)=', header_temp)
                if flag:
                    header = header_temp
                    flag = False
                    continue
                if len(header_temp) < len(header):
                    header = header_temp
        # Parse the temp file and store the attribute values corresponding to the headers obtained
        tree = ElementTree.parse('.temp.txt')
        xml_data = tree.getroot()
        table = []
        for elem in xml_data:
            row = {}
            for j in header:
                try:
                    col = str(f'{header.index(j):02d}')+'_'+j
                    row[col] = elem.attrib[j]
                except:
                    continue
            table.append(row)
        df_table = pd.DataFrame(table)
        df_table.columns = [i[3:] for i in df_table.columns]
        df_table.to_sql(tablename, con=conn, if_exists='append', index=False)
        # Move the finished files to a different folder
        os.system('mv ' + filename + ' ' + pathname +
                  '/../backup/' + pathname.split('/')[-1])


if __name__ == '__main__':
    main()
```

Note that the minimum number of columns were selected, i.e. the attributes that aren't generally found among the posts were ignored, e.g. `ParentId` which is contingent on being an answer post (since the parent must be a question post) or `ClosedDate` which appears only if the question is closed, will not be stored as a column entry in the database. The attributes chosen to be stored into the database are only those that are common or are relevant to the analysis.

A sample command line code used to process the files in the folder `Posts` and store the attributes of the xml files within that to the file `stackoverflow.db` is:

In [2]:
!nohup python projdbload.py pathdbload Posts PostsQuestions stackoverflow.db --filt PostTypeId=\"1\" &

The --filt option was used to filter the posts such that the PostTypeId is equal to 1, corresponding to the questions (contrary to the PostTypeId=2 corresponding to answers).

After storing all the files into the database `stackoverflow.db`, SQL queries can be performed to obtain the necessary data.

In [7]:
conn = sqlite3.connect(
    '/mnt/processed/private/msds2020/lt13/database/stackoverflow.db')
cur = conn.cursor()

cur.execute(f"""SELECT DISTINCT * FROM PostsQuestions 
            WHERE tags LIKE "%python%"
            LIMIT 10""")

names = list(map(lambda x: x[0], cur.description))
pd.DataFrame(cur.fetchall(), columns=names).head()

,Id,PostTypeId,AcceptedAnswerId,CreationDate,Score,ViewCount,Body,OwnerUserId,LastEditorUserId,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,CommunityOwnedDate
0,337,1,342,2008-08-02T03:35:55.697,67,7625,<p>I am about to build a piece of a project th...,111,2336654,2016-12-30T12:56:21.493,2019-05-22T00:27:38.800,XML Processing in Python,<python><xml>,12,1,7,None
1,469,1,3040,2008-08-02T15:11:16.430,38,2655,<p>I am using the Photoshop's javascript API t...,147,1997093,2016-12-22T03:53:45.467,2016-12-22T03:53:45.467,How can I find the full path to a font from it...,<python><macos><fonts><photoshop>,4,0,None,None
2,502,1,7090,2008-08-02T17:01:58.500,40,13577,<p>I have a cross-platform (Python) applicatio...,147,63550,2011-04-08T11:42:03.807,2016-03-25T13:53:13.470,Get a preview JPEG of a PDF on Windows?,<python><windows><image><pdf>,3,0,13,None
3,535,1,541,2008-08-02T18:43:54.787,54,8671,<p>I am starting to work on a hobby project wi...,154,7232508,2018-05-14T17:46:14.650,2018-05-14T17:46:14.650,Continuous Integration System for a Python Cod...,<python><continuous-integration><extreme-progr...,7,0,13,None
4,594,1,595,2008-08-03T01:15:08.507,39,42723,<p>There are several ways to iterate over a re...,116,116,2016-10-14T18:15:27.420,2016-10-15T20:47:11.027,cx_Oracle: How do I iterate over a result set?,<python><sql><database><oracle><cx-oracle>,3,1,8,None


### Appendix A.2. Web Scraping
While the data from the stackexchange archive is complete, the project requires web scraping. And so, the top stackoverflow questions were scraped, particularly the information stored in the XML file.

Some preliminaries before scraping are the proxy settings and the request header to avoid being blocked and minimize the damage if you get blocked.

In [19]:
# Proxy servers for web scraping

os.environ['HTTP_PROXY'] = 'http://3.112.188.39:8080'
os.environ['HTTPS_PROXY'] = 'https://3.112.188.39:8080'

def get_links(url, headers=None):
    """Get the links of the stackoverflow questions under `url`
    """
    resp = requests.get(url, headers=headers)

    resp_soup = BeautifulSoup(resp.text, 'lxml')

    links = [i.get('href')
             for i in resp_soup.select('div.summary a.question-hyperlink')]
    return links

# Request headers

req_header = '''accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3
accept-encoding: gzip, deflate, br
accept-language: en-US,en;q=0.9
cache-control: max-age=0
cookie: _ga=GA1.2.1223690127.1554361835; __qca=P0-1087024619-1554361834541; __gads=ID=95575f8f21b13b6a:T=1554361834:S=ALNI_MYzrAP4MO9xO3-RuodMxFKsUvXijA; notice-ctt=4%3B1554364020909; prov=5a1477ef-3607-2d87-c83b-4dadc0666ec0; _gid=GA1.2.229546230.1560849365; acct=t=sHvGyo1FYK9EJrJ2RTE92i%2bijUmMLbgd&s=ODXb%2b%2f3uMcMKfe3z1cbrgAlLUsX%2bAdVM; _gat=1
referer: https://stackoverflow.com/
upgrade-insecure-requests: 1
user-agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'''

headers = {i.lower(): j for i, j in re.findall(
    r'(.*?): (.*?)$', req_header, re.M)}  # re.M --> multiline
url = 'https://stackoverflow.com/?tab=month'

links = get_links(url, headers)

In [9]:
def scrape_stackoverflow(links, t=5):
    """Scrape each link in the list of links for the attributes stored in the
    XML file from stackexchange archive. An optional parameter t sets the 
    sleep time after each iteration.
    """
    data = []
    for link in links:
        url_2 = 'https://stackoverflow.com/' + link
        retry_count = 0
        while retry_count <= 20:  # Limit the number of retries
            try:
                resp_2 = requests.get(url_2, headers=headers)
                if resp_2.status_code == 200:
                    break
                else:
                    print("Red status code")
                    time.sleep(5)
                    retry_count += 1
            except Exception as e:
                print("Pause for 5 seconds before requesting again")
                time.sleep(5)
                retry_count += 1
        else:
            print("Too many retries")
            break

        # Scrape the data
        resp_soup2 = BeautifulSoup(resp_2.text, 'lxml')

        title = resp_soup2.select_one('div#question-header > h1 > a').text
        tags = [i.text for i in resp_soup2.select(
            'div.post-taglist > div > a')]
        tags = ''.join([f'<{i}>' for i in tags])

        # post id
        row_ID = int(re.findall(r'/questions/(\d+)/', link)[0])
        post_type_ID = 1
        accepted_ans_ID = resp_soup2.findAll(
            'div', {'itemprop': 'acceptedAnswer'})
        if accepted_ans_ID:
            accepted_ans_ID = accepted_ans_ID[0]['data-answerid']
        else:
            accepted_ans_ID = ''

        # post statistics
        question_stats = resp_soup2.select('div.module.question-stats b')
        creation_date = question_stats[0].time['datetime']
        view_count = int(re.findall(
            r'[0-9,]+', question_stats[1].text)[0].replace(',', ''))
        last_act_date = ''
        if len(question_stats) == 3:
            last_act_date = question_stats[2].a['title'].replace(
                ' ', 'T').replace('Z', '')

        #score and body
        score = int(resp_soup2.select_one(
            'div.js-vote-count.grid--cell.fc-black-500.fs-title.grid.fd-column.ai-center').text)
        body = ''.join(list(map(str, resp_soup2.select('div.post-text p'))))

        # user id and editor id
        user_dets_main = resp_soup2.select_one(
            'div.mt16.pt4.grid.gs8.gsy.fw-wrap.jc-end.ai-start')
        user_dets = user_dets_main.select('div.user-details')
        owner_ID = re.findall(r'/users/(\d+)/', user_dets[-1].a['href'])[0]
        if len(user_dets) == 2:
            edit_date = resp_soup2.select(
                'div.question div.user-action-time')[0].span['title'].replace(' ', 'T').replace('Z', '')
            if user_dets[0].has_attr('itemprop'):
                editor_ID = owner_ID
            else:
                try:
                    editor_ID = re.findall(
                        r'/users/(\d+)/', user_dets[0].a['href'])[0]
                except TypeError:
                    editor_ID = ''
        else:
            edit_date = ''
            editor_ID = ''

        # answer and comment count
        ans_count = int(resp_soup2.select_one(
            'div.subheader.answers-subheader').h2['data-answercount'])
        comm = resp_soup2.select('div.question div.comments ul')
        if comm[0]['class'][0] == 'close-as-off-topic-status-list':
            comm.pop(0)
        comm_hidden = int(comm[0][
                          'data-remaining-comments-count'])
        comm_shown = len(resp_soup2.select('div.question span.comment-copy'))
        comm_count = comm_hidden + comm_shown

        data.append((row_ID, post_type_ID, accepted_ans_ID, creation_date,
                     score, view_count, body, owner_ID, editor_ID, edit_date,
                     last_act_date, title, tags, ans_count, comm_count))
        time.sleep(t)
        clear_output()

        count += 1
        links.remove(link)
    return data

In [23]:
data = scrape_stackoverflow(links, t=5)
df = pd.DataFrame(data, columns=['row Id', 'PostTypeId', 'AcceptedAnswerId',
                                 'CreationDate', 'Score', 'ViewCount', 'Body',
                                 'OwnerUserId', 'LastEditorUserId', 'LastEditDate',
                                 'LastActivityDate', 'Title', 'Tags', 'AnswerCount',
                                 'CommentCount']).astype(str)
df.head()

,row Id,PostTypeId,AcceptedAnswerId,CreationDate,Score,ViewCount,Body,OwnerUserId,LastEditorUserId,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount
0,56738380,1,56738667,2019-06-24T14:08:14,110,12753,<p>Since C++ 17 one can write an <code>if</cod...,4850111,63550,2019-06-26T17:36:50,2019-07-09T13:34:20,Most elegant way to write a one-shot 'if',<c++><if-statement><c++17>,8,14
1,56807112,1,56812838,2019-06-28T12:38:46,82,7707,<p>Today I started to receive this error with ...,5790492,1032372,2019-06-28T18:42:18,2019-07-08T14:14:43,Xcode ERROR ITMS-90783: “Missing bundle displa...,<xcode><testflight><fastlane><appstoreconnect>,7,1
2,56866458,1,56866529,2019-07-03T08:58:55,59,4166,<p>I read that in C++17 we can initialize vari...,10858827,3628942,2019-07-10T10:23:55,2019-07-10T10:27:18,Initializing variables in an “if” statement,<c++><c++17>,6,11
3,56692117,1,56692435,2019-06-20T18:43:16,124,6596,"<p>When using the same code, simply changing t...",9419412,63550,2019-06-27T12:21:35,2019-06-27T12:21:35,Why is C++ initial allocation so much larger t...,<c++><c><benchmarking>,2,11
4,56642369,1,56642520,2019-06-18T05:40:29,34,4973,<p>I updated 'android.support:appcompat-v7' to...,11212074,11212074,2019-07-10T14:09:29,2019-07-10T14:09:29,Android Material and appcompat Manifest merger...,<android><react-native><react-native-android><...,13,6
